# SageMaker Data Wrangler Job Notebook
This notebook uses the Data Wrangler .flow file to submit a SageMaker Data Wrangler Job
with the following steps:

* Push Data Wrangler .flow file to S3
* Parse the .flow file inputs, and create the argument dictionary to submit to a boto client
* Submit the ProcessingJob arguments and wait for Job completion

Optionally, the notebook also gives an example of starting a SageMaker XGBoost TrainingJob using
the newly processed data.

In [1]:
%pip install sagemaker==2.16.4 --user

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
import time
import uuid

import boto3
import sagemaker

## Parameters
The following lists configurable parameters that are used throughout this notebook.

In [5]:
# S3 bucket for saving processing job outputs
# Feel free to specify a different bucket here if you wish.
sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "data_wrangler_flows"
flow_id = f"{time.strftime('%d-%H-%M-%S', time.gmtime())}-{str(uuid.uuid4())[:8]}"
flow_name = f"flow-{flow_id}"
flow_uri = f"s3://{bucket}/{prefix}/{flow_name}.flow"

flow_file_name = "99_transform_star_rating_to_sentiment.flow"

iam_role = sagemaker.get_execution_role()

container_uri = "663277389841.dkr.ecr.us-east-1.amazonaws.com/sagemaker-data-wrangler-container:1.0.1"

# Processing Job Resources Configurations
# Data wrangler processing job only supports 1 instance.
instance_count = 1
instance_type = "ml.m5.4xlarge"

# Processing Job Path URI Information
output_prefix = f"export-{flow_name}/output"
output_path = f"s3://{bucket}/{output_prefix}"
output_name = "2b17c16a-8c82-432a-9fec-11ed0b510539.default"

processing_job_name = f"data-wrangler-flow-processing-{flow_id}"

processing_dir = "/opt/ml/processing"

# Modify the variable below to specify the content type to be used for writing each output
# Currently supported options are 'CSV' or 'PARQUET', and default to 'CSV'
output_content_type = "CSV"

# URL to use for sagemaker client.
# If this is None, boto will automatically construct the appropriate URL to use
# when communicating with sagemaker.
sagemaker_endpoint_url = None

## Push Flow to S3
Use the following cell to upload the Data Wrangler .flow file to Amazon S3 so that
it can be used as an input to the processing job.

In [6]:
# Load .flow file
with open(flow_file_name) as f:
    flow = json.load(f)

# Upload to S3
s3_client = boto3.client("s3")
s3_client.upload_file(flow_file_name, bucket, f"{prefix}/{flow_name}.flow")

print(f"Data Wrangler Flow notebook uploaded to {flow_uri}")

Data Wrangler Flow notebook uploaded to s3://sagemaker-us-east-1-835319576252/data_wrangler_flows/flow-03-00-04-50-3d942e17.flow


## Create boto3 Processing Job arguments

This notebook submits a Processing Job using boto, which will require an argument dictionary to
submit to the boto client. Below, utility methods are defined for creating Processing Job Inputs
for the following sources: S3, Athena, and Redshift. Then the argument dictionary is generated
using the parsed inputs and job configurations such as instance type.

In [7]:
def create_flow_notebook_processing_input(base_dir, flow_s3_uri):
    return {
        "InputName": "flow",
        "S3Input": {
            "LocalPath": f"{base_dir}/flow",
            "S3Uri": flow_s3_uri,
            "S3DataType": "S3Prefix",
            "S3InputMode": "File",
        },
    }

def create_s3_processing_input(base_dir, name, dataset_definition):
    return {
        "InputName": name,
        "S3Input": {
            "LocalPath": f"{base_dir}/{name}",
            "S3Uri": dataset_definition["s3ExecutionContext"]["s3Uri"],
            "S3DataType": "S3Prefix",
            "S3InputMode": "File",
        },
    }

def create_redshift_processing_input(base_dir, name, dataset_definition):
    return {
        "InputName": name,
        "DatasetDefinition": {
            "RedshiftDatasetDefinition": {
                "ClusterId": dataset_definition["clusterIdentifier"],
                "Database": dataset_definition["database"],
                "DbUser": dataset_definition["dbUser"],
                "QueryString": dataset_definition["queryString"],
                "ClusterRoleArn": dataset_definition["unloadIamRole"],
                "OutputS3Uri": f'{dataset_definition["s3OutputLocation"]}{name}/',
                "OutputFormat": dataset_definition["outputFormat"].upper(),
            },
            "LocalPath": f"{base_dir}/{name}",
        },
    }

def create_athena_processing_input(base_dir, name, dataset_definition):
    return {
        "InputName": name,
        "DatasetDefinition": {
            "AthenaDatasetDefinition": {
                "Catalog": dataset_definition["catalogName"],
                "Database": dataset_definition["databaseName"],
                "QueryString": dataset_definition["queryString"],
                "OutputS3Uri": f'{dataset_definition["s3OutputLocation"]}{name}/',
                "OutputFormat": dataset_definition["outputFormat"].upper(),
            },
            "LocalPath": f"{base_dir}/{name}",
        },
    }

def create_processing_inputs(processing_dir, flow, flow_uri):
    """Helper function for creating processing inputs
    :param flow: loaded data wrangler flow notebook
    :param flow_uri: S3 URI of the data wrangler flow notebook
    """
    processing_inputs = []
    flow_processing_input = create_flow_notebook_processing_input(processing_dir, flow_uri)
    processing_inputs.append(flow_processing_input)

    for node in flow["nodes"]:
        if "dataset_definition" in node["parameters"]:
            data_def = node["parameters"]["dataset_definition"]
            name = data_def["name"]
            source_type = data_def["datasetSourceType"]

            if source_type == "S3":
                s3_processing_input = create_s3_processing_input(
                    processing_dir, name, data_def)
                processing_inputs.append(s3_processing_input)
            elif source_type == "Athena":
                athena_processing_input = create_athena_processing_input(
                    processing_dir, name, data_def)
                processing_inputs.append(athena_processing_input)
            elif source_type == "Redshift":
                redshift_processing_input = create_redshift_processing_input(
                    processing_dir, name, data_def)
                processing_inputs.append(redshift_processing_input)
            else:
                raise ValueError(f"{source_type} is not supported for Data Wrangler Processing.")
    return processing_inputs

def create_container_arguments(output_name, output_content_type):
    output_config = {
        output_name: {
            "content_type": output_content_type
        }
    }
    return [f"--output-config '{json.dumps(output_config)}'"]

# Create Processing Job Arguments
processing_job_arguments = {
    "AppSpecification": {
        "ContainerArguments": create_container_arguments(output_name, output_content_type),
        "ImageUri": container_uri,
    },
    "ProcessingInputs": create_processing_inputs(processing_dir, flow, flow_uri),
    "ProcessingOutputConfig": {
        "Outputs": [
            {
                "OutputName": output_name,
                "S3Output": {
                    "S3Uri": output_path,
                    "LocalPath": os.path.join(processing_dir, "output"),
                    "S3UploadMode": "EndOfJob",
                }
            },
        ],
    },
    "ProcessingJobName": processing_job_name,
    "ProcessingResources": {
        "ClusterConfig": {
            "InstanceCount": instance_count,
            "InstanceType": instance_type,
            "VolumeSizeInGB": 30,
        }
    },
    "RoleArn": iam_role,
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 86400,
    },
}

## Start ProcessingJob
Now, the Processing Job is submitted to a boto client. The status of the processing job is
monitored with the boto client, and this notebook waits until the job is no longer 'InProgress'.

In [8]:
sagemaker_client = boto3.client("sagemaker", endpoint_url=sagemaker_endpoint_url)
create_response = sagemaker_client.create_processing_job(**processing_job_arguments)

status = sagemaker_client.describe_processing_job(ProcessingJobName=processing_job_name)

In [10]:
while status["ProcessingJobStatus"] == "InProgress":
    status = sagemaker_client.describe_processing_job(ProcessingJobName=processing_job_name)
    print(status["ProcessingJobStatus"])
    time.sleep(60)

print(status)

InProgress
InProgress
InProgress
InProgress
InProgress
Completed
{'ProcessingInputs': [{'InputName': 'flow', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/data_wrangler_flows/flow-03-00-04-50-3d942e17.flow', 'LocalPath': '/opt/ml/processing/flow', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated'}}, {'InputName': 'amazon-reviews-pds-tsv', 'AppManaged': False, 'DatasetDefinition': {'AthenaDatasetDefinition': {'Catalog': 'AwsDataCatalog', 'Database': 'dsoaws', 'QueryString': 'select * from amazon_reviews_tsv', 'OutputS3Uri': 's3://sagemaker-us-east-1-835319576252/athena/amazon-reviews-pds-tsv/', 'OutputDatasetS3Uri': 's3://sagemaker-us-east-1-835319576252/athena/amazon-reviews-pds-tsv/data-wrangler-flow-processing-03-00-04-50-3d942e17/data', 'OutputFormat': 'PARQUET'}, 'LocalPath': '/opt/ml/processing/amazon-reviews-pds-tsv', 'DataDistributionType': 'ShardedByS3Key', 'InputMode': 'File'}}], 'ProcessingOutputCon

In [18]:
!aws s3 ls s3://sagemaker-us-east-1-835319576252/export-flow-03-00-04-50-3d942e17/output/data-wrangler-flow-processing-03-00-04-50-3d942e17/2b17c16a-8c82-432a-9fec-11ed0b510539/default/



2020-12-03 00:09:47          0 _SUCCESS
2020-12-03 00:09:36        238 part-00000-89e06de2-4fbe-4477-85e7-10121e5e2153-c000.csv
2020-12-03 00:09:38   55877158 part-00002-89e06de2-4fbe-4477-85e7-10121e5e2153-c000.csv
2020-12-03 00:09:37   14346687 part-00004-89e06de2-4fbe-4477-85e7-10121e5e2153-c000.csv
2020-12-03 00:09:38   58569394 part-00007-89e06de2-4fbe-4477-85e7-10121e5e2153-c000.csv
2020-12-03 00:09:36      91073 part-00012-89e06de2-4fbe-4477-85e7-10121e5e2153-c000.csv
2020-12-03 00:09:36      87665 part-00013-89e06de2-4fbe-4477-85e7-10121e5e2153-c000.csv


In [21]:
!aws s3 cp s3://sagemaker-us-east-1-835319576252/export-flow-03-00-04-50-3d942e17/output/data-wrangler-flow-processing-03-00-04-50-3d942e17/2b17c16a-8c82-432a-9fec-11ed0b510539/default/part-00012-89e06de2-4fbe-4477-85e7-10121e5e2153-c000.csv ./

download: s3://sagemaker-us-east-1-835319576252/export-flow-03-00-04-50-3d942e17/output/data-wrangler-flow-processing-03-00-04-50-3d942e17/2b17c16a-8c82-432a-9fec-11ed0b510539/default/part-00012-89e06de2-4fbe-4477-85e7-10121e5e2153-c000.csv to ./part-00012-89e06de2-4fbe-4477-85e7-10121e5e2153-c000.csv


In [22]:
!cat part-00012-89e06de2-4fbe-4477-85e7-10121e5e2153-c000.csv

marketplace,review_id,product_id,product_title,product_category,vine,verified_purchase,review_headline,review_body,review_date,customer_id,product_parent,star_rating,helpful_votes,total_votes,star_rating_scaled,star_rating_scaled_floored
US,R328E814YY64A4,B004RMK4P8,Playstation Network Card,Digital_Video_Games,N,Y,Digital and Fast,Flawless! Recommend! Save my Life and open the doors of playstation network for me and my friends. Thank you so much!,2012-11-29,9628723,384246568,5,0,0,1.0,1
US,R3C4A7JFJ3YK9T,B008Y218RK,Lord of the Rings: War in the North [Download],Digital_Video_Games,N,Y,"When on sale, a great buy","I bought it on sale, so I don't think I would spend over $10 on it, but it is pretty fun. Like a mix between LOTR: Third Age and the Return of the King game",2012-11-29,48034084,102056685,4,0,0,0.5,0
US,R1LC2JJITDDGA5,B008EMXQPM,Raptor : Call of the Shadow - 2010 Edition [Download],Digital_Video_Games,N,Y,AWESOME GAME,"I ALWAYS LOVED THIS GAME AND WAS SO HAPPY THAT I FINALLY F

## Kick off SageMaker Training Job (Optional)
Data Wrangler is a SageMaker tool for processing data to be used for Machine Learning. Now that
the data has been processed, users will want to train a model using the data. The following shows
an example of doing so using a popular algorithm XGBoost.

It is important to note that the following XGBoost objective ['binary', 'regression',
'multiclass'], hyperparameters, or content_type may not be suitable for the output data, and will
require changes to train a proper model. Furthermore, for CSV training, the algorithm assumes that
the target variable is in the first column. For more information on SageMaker XGBoost, please see
https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html.

### Find Training Data path
The below demonstrates how to recursively search the output directory to find the data location.

In [11]:
s3_client = boto3.client("s3")
list_response = s3_client.list_objects_v2(Bucket=bucket, Prefix=output_prefix)

training_path = None

for content in list_response["Contents"]:
    if "_SUCCESS" not in content["Key"]:
        training_path = content["Key"]

print(training_path)

export-flow-03-00-04-50-3d942e17/output/data-wrangler-flow-processing-03-00-04-50-3d942e17/2b17c16a-8c82-432a-9fec-11ed0b510539/default/part-00013-89e06de2-4fbe-4477-85e7-10121e5e2153-c000.csv


Next, the Training Job hyperparameters are set. For more information on XGBoost Hyperparameters,
see https://xgboost.readthedocs.io/en/latest/parameter.html.

In [12]:
region = boto3.Session().region_name
container = sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")
hyperparameters = {
    "max_depth":"5",
    "objective": "reg:squarederror",
    "num_round": "10",
}
train_content_type = (
    "application/x-parquet" if output_content_type.upper() == "PARQUET"
    else "text/csv"
)
train_input = sagemaker.inputs.TrainingInput(
    s3_data=f"s3://{bucket}/{training_path}",
    content_type=train_content_type,
)

The TrainingJob configurations are set using the SageMaker Python SDK Estimator, and which is fit
using the training data from the ProcessingJob that was run earlier.

In [ ]:
estimator = sagemaker.estimator.Estimator(
    container,
    iam_role,
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
)
estimator.fit({"train": train_input})